In [1]:
# !pip install -q -r requirements.txt')
# !pip install -q stanza
from adjective_reading.parsing import *

In [2]:
odf = pd.DataFrame(iter_parsed_sents_stats('reading', limit=12_000))

 13%|█▎        | 9336/71902 [00:52<05:55, 176.22it/s]


In [142]:
odf2=odf.set_index(['url','sent_id','sent_str','id','head_id','label']).sort_index()
odf2

sent_repr  \
url                                        sent_id sent_str                                           id head_id label                                                                            
http://www.jstor.org/stable/10.1086/380806 6123    Doing so I promote a reading that, however tena... 6  4       obj(promote, reading)        Doing so I PROMOTE a READING that, however ten...   
                                                                                                         5       det(a, reading)              Doing so I promote A READING that, however ten...   
                                                                                                         14      acl:relcl(wrong, reading)    Doing so I promote a READING that, however ten...   
                                           6124    Such a reading advertises tantalizingly concret... 3  1       det:predet(Such, reading)    SUCH a READING advertises tantalizingly concre...   
                                                                                                         2       det(a, reading)              Such A READING advertises tantalizingly concre...   
...                                                                                                                                                                                         ...   
http://www.jstor.org/stable/823189         3009    It became apparent early on that unless the dep... 31 34      compound(skills, reading)    It became apparent early on that unless the de...   
http://www.jstor.org/stable/823270         4686    As we shall see, this point holds certain conse... 13 10      nmod(consequences, reading)  As we shall see, this point holds certain CONS...   
                                                                                                         11      case(for, reading)           As we shall see, this point holds certain cons...   
                                                                                                         12      det(any, reading)            As we shall see, this point holds certain cons...   
                                                                                                         17      nmod(poem, reading)          As we shall see, this point holds certain cons...   

                                                                                                                                                 head_text  \
url                                        sent_id sent_str                                           id head_id label                                       
http://www.jstor.org/stable/10.1086/380806 6123    Doing so I promote a reading that, however tena... 6  4       obj(promote, reading)             promote   
                                                                                                         5       det(a, reading)                         a   
                                                                                                         14      acl:relcl(wrong, reading)           wrong   
                                           6124    Such a reading advertises tantalizingly concret... 3  1       det:predet(Such, reading)            Such   
                                                                                                         2       det(a, reading)                         a   
...                                                                                                                                                    ...   
http://www.jstor.org/stable/823189         3009    It became apparent early on that unless the dep... 31 34      compound(skills, reading)          skills   
http://www.jstor.org/stable/823270         4686    As we shall see, this point holds certain conse... 13 10      nmod(consequences, reading)  consequences   
                                                                  

In [4]:
odf.drop_duplicates(subset=['url','sent_str','id','head_id'])

,url,sent_id,sent_str,sent_repr,id,head_id,head_text,head_rel,head_type,label,text
0,http://www.jstor.org/stable/461288,0,We are all our lifetime reading the copious se...,We are all our LIFETIME READING the copious se...,6,5,lifetime,acl,head,"acl(lifetime, reading)",reading
1,http://www.jstor.org/stable/461288,0,We are all our lifetime reading the copious se...,We are all our lifetime READING the copious SE...,6,9,sense,obj,dep,"obj(sense, reading)",reading
2,http://www.jstor.org/stable/40587680,1,This suggests the significance of logical proc...,This suggests the significance of logical proc...,29,26,one,acl:relcl,head,"acl:relcl(one, reading)",reading
3,http://www.jstor.org/stable/40587680,1,This suggests the significance of logical proc...,This suggests the significance of logical proc...,29,25,whether,mark,dep,"mark(whether, reading)",reading
4,http://www.jstor.org/stable/40587680,1,This suggests the significance of logical proc...,This suggests the significance of logical proc...,29,27,they,nsubj,dep,"nsubj(they, reading)",reading
...,...,...,...,...,...,...,...,...,...,...,...
30389,http://www.jstor.org/stable/1344089,11996,We need to go beyond a purely semiological rea...,We need to go beyond a purely semiological REA...,13,15,such,nmod,dep,"nmod(such, reading)",reading
30390,http://www.jstor.org/stable/1344089,11997,"In some cases, the inscription inside, reading...","In some cases, the INSCRIPTION inside, READING...",9,6,inscription,appos,head,"appos(inscription, reading)",reading
30391,http://www.jstor.org/stable/457186,11999,Thus a popular geste grew into a whole cycle o...,Thus a popular geste grew into a whole cycle o...,35,33,became,xcomp,head,"xcomp(became, reading)",reading
30392,http://www.jstor.org/stable/457186,11999,Thus a popular geste grew into a whole cycle o...,Thus a popular geste grew into a whole cycle o...,35,34,staple,amod,dep,"amod(staple, reading)",reading


In [5]:
url2year = {d['url']:d['publicationYear'] for d in orjsonl.stream(PATH_CORPUS)}
url2decade = {url:f"{year//10*10}-{year//10*10+9}" for url,year in url2year.items()}

from collections import defaultdict
decade2urls = defaultdict(list)
for url,decade in url2decade.items():
    decade2urls[decade].append(url)

# decade2urls

In [9]:
min(url2year.values())

1900

In [24]:
def periodize_3way(y):
    if y<1968:
        return '1920-1968'
    elif y<2000:
        return '1968-2000'
    else:
        return '2000-2018'

def periodize(y, cutoffs=[1920,1945,1970,1995], include_before=False, include_after=False):
    cutoffs.sort()
    mincut = cutoffs[0]
    maxcut = cutoffs[-1]
    if y < mincut:
        return f'{mincut}-{cutoffs[0]}' if include_before else f''
    elif y > maxcut:
        return f'{cutoffs[-1]}-' if include_after else f''

    for i in range(len(cutoffs)-1):
        if y < cutoffs[i+1]:
            return f'{cutoffs[i]}-{cutoffs[i+1]}'
    return f'{cutoffs[-1]}-' if include_after else f''

    # if y < cutoffs[0]:
    #     return f'{cutoffs[0]}-{cutoffs[1]}  '
    # elif y < cutoffs[1]:
    #     return f'{cutoffs[1]}-{cutoffs[2]}'
    # elif y < cutoffs[2]:
    #     return f'{cutoffs[2]}-{cutoffs[3]}'
    # else:
    #     return f'{cutoffs[3]}-{cutoffs[4]}'


# odf['year'] = odf['url'].map(url2year)
# odf['decade'] = odf['year'] // 10 * 10
# odf['period'] = odf['year'].map(periodize)


In [28]:
periodize(2020, cutoffs=[1920, 1950, 1980, 2000, 2020])

''

In [130]:
PERIOD_CUTOFFS = [1920, 1960, 1980, 1990, 2000, 2010, 2020]

odf['period'] = odf['year'].map(lambda y: periodize(y, cutoffs=PERIOD_CUTOFFS))
odf.query('period!=""').period.value_counts().sort_index()

period
1920-1960    4017
1960-1980    4380
1980-1990    5203
1990-2000    6124
2000-2010    5214
2010-2020    4614
Name: count, dtype: int64

In [79]:
odf.iloc[0]

url                         http://www.jstor.org/stable/461288
sent_id                                                      0
sent_str     We are all our lifetime reading the copious se...
sent_repr    We are all our LIFETIME READING the copious se...
id                                                           6
head_id                                                      5
head_text                                             lifetime
head_rel                                                   acl
head_type                                                 head
label                                   acl(lifetime, reading)
text                                                   reading
year                                                      1967
decade                                                    1960
period                                               1960-1980
Name: 0, dtype: object

In [131]:
odfx = odf.query('period!=""').copy()
odfx['period_int'] = odfx['period'].map(lambda x: int(x.split('-')[0]))
odfx = odfx[odfx.text.str.lower() == 'reading']
odfx['key'] = list(zip(odfx['head_rel'], odfx['head_text'].str.lower()))

In [132]:
min_period_count = odfx.period.value_counts().min()
min_period_countK = min_period_count // 1000 * 1000
odfx_smpl = odfx.groupby('period').sample(min_period_count)
odfx_smpl.period.value_counts()

period
1920-1960    3980
1960-1980    3980
1980-1990    3980
1990-2000    3980
2000-2010    3980
2010-2020    3980
Name: count, dtype: int64

In [133]:
odfx_smpl.text.value_counts()

text
reading    21967
Reading     1680
READING      233
Name: count, dtype: int64

In [135]:
counts_df = odfx.groupby('period').key.value_counts().sort_values(ascending=False).reset_index()
counts_df['period_int'] = counts_df['period'].map(lambda x: int(x.split('-')[0]))
counts_df['head_rel'], counts_df['head_text'] = zip(*counts_df['key'])
counts_df

,period,key,count,period_int,head_rel,head_text
0,1990-2000,"(det, a)",274,1990,det,a
1,1990-2000,"(case, of)",268,1990,case,of
2,1980-1990,"(det, a)",238,1980,det,a
3,1920-1960,"(det, the)",230,1920,det,the
4,2000-2010,"(case, of)",220,2000,case,of
...,...,...,...,...,...,...
14883,1980-1990,"(nmod, comique)",1,1980,nmod,comique
14884,1980-1990,"(nmod, conceptions)",1,1980,nmod,conceptions
14885,1980-1990,"(nmod, conclusion)",1,1980,nmod,conclusion
14886,1980-1990,"(nmod, conflict)",1,1980,nmod,conflict


In [136]:
counts_df.period.value_counts().sort_index()

period
1920-1960    2041
1960-1980    2180
1980-1990    2542
1990-2000    3013
2000-2010    2681
2010-2020    2431
Name: count, dtype: int64

In [137]:
min_count_period = counts_df.period.value_counts().min()
counts_df_smpl = counts_df.groupby('period').sample(min_count_period // 10 * 10)
counts_df_smpl.period.value_counts()

period
1920-1960    2040
1960-1980    2040
1980-1990    2040
1990-2000    2040
2000-2010    2040
2010-2020    2040
Name: count, dtype: int64

In [138]:
counts_df_smpl.sort_values('count', ascending=False).head(10)

,period,key,count,period_int,head_rel,head_text
0,1990-2000,"(det, a)",274,1990,det,a
1,1990-2000,"(case, of)",268,1990,case,of
2,1980-1990,"(det, a)",238,1980,det,a
3,1920-1960,"(det, the)",230,1920,det,the
5,2010-2020,"(case, of)",217,2010,case,of
6,2000-2010,"(det, a)",203,2000,det,a
7,1960-1980,"(det, a)",196,1960,det,a
8,1980-1990,"(case, of)",194,1980,case,of
9,2010-2020,"(det, a)",190,2010,det,a
10,1980-1990,"(det, the)",160,1980,det,the


In [139]:
counts_df_smpl.to_excel('nlp_df_smpl.xlsx')

In [146]:
odf.query('head_text == "symptomatic"')

,url,sent_id,sent_str,sent_repr,id,head_id,head_text,head_rel,head_type,label,text,year,decade,period
7242,http://www.jstor.org/stable/24475529,2950,Stephen Best and Sharon Marcus have recently a...,Stephen Best and Sharon Marcus have recently a...,19,18,symptomatic,amod,dep,"amod(symptomatic, reading)",reading,2013,2010,2010-2020
13776,http://www.jstor.org/stable/26547658,5509,in his explorations of the conceptual logic of...,in his explorations of the conceptual logic of...,51,50,symptomatic,amod,dep,"amod(symptomatic, reading)",reading,2016,2010,2010-2020
22030,http://www.jstor.org/stable/10.1086/592539,8819,The Marxist symptomatic reading can convincing...,The Marxist SYMPTOMATIC READING can convincing...,4,3,symptomatic,amod,dep,"amod(symptomatic, reading)",reading,2008,2000,2000-2010
